### With this piece of code found on Github I managed to retrieve historical data from Reddit by calling the Pushshift API and by setting the two timestamps of the time range 
I used mainly this method to retrieve data for our control group. 

In [1]:
#import required libraries 
import pandas as pd
import csv
import json
import os
import requests  
import time
import datetime

In [ ]:
def getPushshiftData(before, after, sub):
    import time
    #Build URL
    url = 'https://api.pushshift.io/reddit/search/submission/?&size=1000&after='+str(after)+'&before='+str(before)+'&subreddit='+str(sub)
    
    #Print URL to show user
    print(url)
    
    #Request URL
    r = requests.get(url)
    
    #Load JSON data from webpage into data variable
    data = json.loads(r.text)
    time.sleep(10)
    #return the data element which contains all the submissions data
    return data['data']

In [ ]:
def collectSubData(subm):
    #subData was created at the start to hold all the data which is then added to our global subStats dictionary.
    subData = list() #list to store data points
    title = subm['title']
    sub_id = subm['id']
    score = subm['score']
    created = datetime.datetime.fromtimestamp(subm['created_utc']) #1520561700.0
    numComms = subm['num_comments']
    selftext = subm['selftext']
    subreddit = subm['subreddit']
    over_18 = subm['over_18']
    #Put all data points into a tuple and append to subData
    subData.append((sub_id,title,selftext,score,created,numComms,over_18,subreddit))
    #Create a dictionary entry of current submission data and store all data related to it
    subStats[sub_id] = subData

In [ ]:
#choose suitable timestamps
before = 1643034976
after = 1640356576
submission = 'IAmA'

#subCount tracks the no. of total submissions we collect
subCount = 0
#subStats is the dictionary where we will store our data.
subStats = {}

In [ ]:
# We need to run this function outside the loop first to get the updates after variable
data = getPushshiftData(after, before, submission)

# Will run until all posts have been gathered i.e. When the length of data variable = 0
# from the 'after' date up until before date

while len(data) > 0: #The length of data is the number submissions (data[0], data[1] etc), once it hits zero (after and before vars are the same) end
    for submission in data:
        try:
            collectSubData(submission)
            subCount+=1
        except:
            continue
            
    # Calls getPushshiftData() with the created date of the last submission
    # print(len(data))
    # print(str(datetime.datetime.fromtimestamp(data[-1]['created_utc'])))
    #update after variable to last created date of submission
    after = data[-1]['created_utc']
    # print("CHECK")
    #data has changed due to the new after variable provided by above code
    try:
        data = getPushshiftData(after, before, sub)
        print(subCount)
    except:
        while 1:
            if int(after) >= int(before):
                break
        try:
            print(str(datetime.datetime.fromtimestamp(data[-1]['created_utc'])))
            after += 1000
            data = getPushshiftData(after, before, submission)
            print(subCount)
            break
        except:
            after += 1000 #Edit with small numbers if you want more posts

print(len(data))

https://api.pushshift.io/reddit/search/submission/?&size=1000&after=1643034976&before=1640356576&subreddit=IAmA


KeyboardInterrupt: 

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
print(str(len(subStats)) + " submissions have added to list")
print("1st entry is:")
print(list(subStats.values())[0][0][1] + " created: " + str(list(subStats.values())[0][0][5]))
print("Last entry is:")
print(list(subStats.values())[-1][0][1] + " created: " + str(list(subStats.values())[-1][0][5]))

In [ ]:
def updateSubs_file():
    upload_count = 0
    #location = "\\Reddit Data\\" >> If you're running this outside of a notebook you'll need this to direct to a specific location
    print("input filename of submission file, please add .csv")
    filename = input() #This asks the user what to name the file
    file = filename
    with open(file, 'w', newline='', encoding='utf-8') as file: 
        a = csv.writer(file, delimiter=',')
        headers = ["Post_iD","Title","Body","Score","Publish_date","Total_no_of_comments", "Over_18", "Subreddit"]
        a.writerow(headers)
        for sub in subStats:
            a.writerow(subStats[sub][0])
            upload_count+=1
            
        print(str(upload_count) + " submissions have been uploaded")
updateSubs_file()

#### Join together all the submissions otained in the four different periods you selected in the previous function. 
Starting date = 1st Jan 2019
Ending date = 22nd Nov 2021

In [4]:
#this is the function I defined to be able to read and write from many input files 
#and add to a single output file all the posts collected

import os

def append_to_file(file_in, file_out):
    lista_files = os.listdir()
    
    #first iteration
    if file_out not in lista_files: 
        with open(file_out, 'a') as f:
            writer = csv.writer(f)
            writer.writerow(['index', 'text', 'class'])
    
            with open(file_in, 'r') as file_in:
                file = csv.reader(file_in, delimiter = ',')
                #skip header
                next(file) 
                i = 0
                for riga in file:
                    text = riga[2]
                    classe = riga[-1]

                    if text != '' and text != '[removed]' and text != '[deleted]':
                        i += 1
                        writer.writerow([i, text, classe])
                        
    else:
        df = pd.read_csv(file_out)
        starting_i = len(df)
        
        with open(file_out, 'a') as f:
            writer = csv.writer(f)
    
            with open(file_in, 'r') as file_in:
                file = csv.reader(file_in, delimiter = ',')
                #skip header
                next(file) 
        
                for riga in file:
                    text = riga[2]
                    classe = riga[-1]
            
                    if text != '' and text != '[removed]' and text != '[deleted]':
                        starting_i += 1
                        writer.writerow([starting_i, text, classe])
                    


In [5]:
#running this cell allows you to otain the final csv for teh control group
#WARNING: don't run it again 
iterator = ['/content/drive/MyDrive/Computational_Linguistics_Project /datasets/control.csv',
            '/content/drive/MyDrive/Computational_Linguistics_Project /datasets/control_2.csv', 
            '/content/drive/MyDrive/Computational_Linguistics_Project /datasets/control_3.csv', 
            '/content/drive/MyDrive/Computational_Linguistics_Project /datasets/control_4.csv', 
            '/content/drive/MyDrive/Computational_Linguistics_Project /datasets/control_5.csv', 
            '/content/drive/MyDrive/Computational_Linguistics_Project /datasets/control_6.csv', 
            '/content/drive/MyDrive/Computational_Linguistics_Project /datasets/changemyview.csv', 
            '/content/drive/MyDrive/Computational_Linguistics_Project /datasets/freethought.csv', 
            '/content/drive/MyDrive/Computational_Linguistics_Project /datasets/yoga.csv', 
            '/content/drive/MyDrive/Computational_Linguistics_Project /datasets/foodforthought.csv']

for f in iterator:
    append_to_file(f, 'df_control_group.csv')
    

In [6]:
#you can open the control dataset, saved in .csv format
control_df = pd.read_csv('df_control_group.csv', index_col= 0)
print(control_df.shape[0])

12041


In [7]:
#open again the .csv file where you want to write data in (just because we were missing some previously collected data)
with open('df_control_group.csv', 'a') as file_app:
    writer = csv.writer(file_app)
    #I list all the files that I have in this directory
    file_dir = os.listdir()   
    for file in file_dir:
        if file == 'IAmA.json':
            classe = file.split('.j')[0]
            f = open(f"{classe}.json")
            data = json.load(f)
            i = control_df.shape[0]
            for post, text in list(data.items()):
                i += 1
                writer.writerow([i, text, classe])
                    
                    

In [8]:
os.listdir()

['.config', 'df_control_group.csv', 'drive', 'sample_data']